In [1]:
import re
#!pip install tensorflow
import tensorflow as tf
#!pip install keras.preprocessing
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
import string
from tensorflow import keras
#!pip install nltk
import nltk
from nltk.corpus import stopwords
#!pip install wordcloud
from wordcloud import WordCloud
import matplotlib.pyplot as plt
#!pip install seaborn
import seaborn as sns
import collections
import numpy as np
import pandas as pd


2022-12-13 17:00:04.877053: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-13 17:00:04.877943: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2022-12-13 17:00:04.877953: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.


In [2]:
#Read in the data
import pandas as pd

df1=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/clean_emails_df1.csv")
df2=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/clean_emails_df2.csv")
df3=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/clean_emails_df3.csv")
df4=pd.read_csv("https://raw.githubusercontent.com/statzenthusiast921/political-emails-analysis/main/main/data/clean_emails_df4.csv")

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)


(19000, 20)
(19000, 20)
(19000, 20)
(17148, 20)


In [3]:
#Stack 4 dataframes together
df = pd.concat([df1, df2, df3, df4], ignore_index=True, axis=0)
#Delete first row and column
df = df[1:]
df = df.iloc[: , 1:]

#Drop NAs from cleaned_body and body
df = df[df['cleaned_body'].notna()]
#Preview dataframe
df.head()


,subject,date,body,party,country,locality,office,time,AM_PM,Hour,Hour_Mil,month,day,year,month_num,cleaned_body,sentiment,compound,comp_score
1,TAKE ACTION for Freedom: #CloseTheCamps,"July 3, 2019","Dear friend, We have all seen the images and r...",NaN,United States,NaN,NaN,11:31 PM,PM,11,23,July,3,2019,7,dear friend seen images read stories migrants ...,"{'neg': 0.207, 'neu': 0.674, 'pos': 0.12, 'com...",-0.9807,0
2,trauma-informed schools,"July 3, 2019","Team,Access to education is fundamental to a c...",Democratic,United States,Ohio,President of the United States,11:32 PM,PM,11,23,July,3,2019,7,team access education fundamental child succes...,"{'neg': 0.143, 'neu': 0.658, 'pos': 0.199, 'co...",0.7269,1
3,Juárez,"July 3, 2019","Beto for America John, The Trump administrati...",Democratic,United States,Texas,President of the United States,11:32 PM,PM,11,23,July,3,2019,7,beto america john trump administration created...,"{'neg': 0.069, 'neu': 0.684, 'pos': 0.247, 'co...",0.9578,1
4,Andy's new veterans jobs plan,"July 4, 2019","John,Tomorrow is the Fourth of July, a day for...",Democratic,United States,Kentucky,State Governor,12:15 AM,AM,12,0,July,4,2019,7,john tomorrow fourth july day us come together...,"{'neg': 0.023, 'neu': 0.636, 'pos': 0.341, 'co...",0.9867,1
5,say NO to Trump's military parade [signature n...,"July 4, 2019","Tomorrow is the Fourth of July, and Donald Tru...",Democratic,United States,NaN,President of the United States,12:19 AM,AM,12,0,July,4,2019,7,tomorrow fourth july donald trump using day th...,"{'neg': 0.078, 'neu': 0.684, 'pos': 0.238, 'co...",0.8316,1


In [4]:
df = df[df['year']>1970]
df['locality'] = np.where(df['locality']=="MIchigan","Michigan",df['locality'])
df.shape

(73694, 19)

In [5]:
nltk.download('stopwords')
stop_words = stopwords.words('english')
# stop_words.extend(["uh","oh","okay","im","dont"])

[nltk_data] Downloading package stopwords to /home/studio-lab-
[nltk_data]     user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
df['body'] = df['body'].astype('string')

In [7]:
def remove_brackets_contents(text):
    pattern = r"\[.*?\]"
    return re.sub(pattern, " ", text)


def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, ' ', text)

In [8]:
import warnings
warnings.filterwarnings('ignore')

df['body'] = df['body'].replace('\n','',regex=True)
df['body'] = df['body'].replace('\r','',regex=True)
df.loc[:,'body'] = df['body'].apply(func = remove_brackets_contents)
df.loc[:,'body'] = df['body'].apply(func = remove_special_characters)


df['body'] = df['body'].replace('   ',' ',regex=True)
df['body'] = df['body'].replace('    ',' ',regex=True)
df['body'] = df['body'].replace('     ',' ',regex=True)

df['body'] = df['body'].replace(']',' ',regex=True)
df['body'] = df['body'].replace('\xa0',' ',regex=True)
df['body'] = df['body'].replace('almostdaily','almost daily',regex=True)


#Remove all the text that follows these sentences
df['body'] = df['body'].str.replace("(If you need to remove yourself from our email).*","")
df['body'] = df['body'].str.replace("(Email is the most important way).*","")
df['body'] = df['body'].str.replace("(This email was sent to).*","")
df['body'] = df['body'].str.replace("(Thank you for speaking).*","")
df['body'] = df['body'].str.replace("(All Rights Reserved).*","")
df['body'] = df['body'].str.replace("(mailto:).*","")
df['body'] = df['body'].str.replace("(Thank you for choosing to fight).*","")
df['body'] = df['body'].str.replace("(Paid for by).*","")
df['body'] = df['body'].str.replace("(We look forward to hearing from you).*","")
df['body'] = df['body'].str.replace("(PAID FOR BY).*","")
df['body'] = df['body'].str.replace("(Thanks for your).*","")
df['body'] = df['body'].str.replace("(All rights reserved).*","")
df['body'] = df['body'].str.replace("(unsubscribe).*","")
df['body'] = df['body'].str.replace("(Sincerely).*","")
df['body'] = df['body'].str.replace("(Facebook).*","")
df['body'] = df['body'].str.replace("(Twitter).*","")
df['body'] = df['body'].str.replace("(Please, sign your name).*","")
df['body'] = df['body'].str.replace("(CONTRIBUTE).*","")
df['body'] = df['body'].str.replace("(Contribute).*","")


In [9]:

from nltk.tokenize import RegexpTokenizer
#!pip install contractions
import contractions

def punct(text):
    token=RegexpTokenizer(r'\w+')#regex
    text = token.tokenize(text)
    text= " ".join(text)
    return text 

def remove_special_characters(text):
    # define the pattern to keep
    pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]' 
    return re.sub(pat, '', text)

def remove_digits(text):
    pattern = r'[^a-zA-z.,!?/:;\"\'\s]' 
    return re.sub(pattern, '', text)

def lemmatize(text):
    lemmed = contractions.fix(str(text))
    return lemmed



def remove_weird_words_that_showed_up_in_analysis_(str):
    #select english stopwords
    #cachedStopWords = set(stopwords.words("english"))
    #add custom words
    cachedStopWords = set(('height','width','style','saved','payment','information','https',
                            'fmailchi','mp','chip','tr','td','style','font','size','px','view'
                            'email','browser','zwnj','metered','paywall','articles','vertical',
                            'align','top','item','img','src','rss','mc','actblue','com','donate',
                            'donation', 'padding','bottom', 'members','pre','unsubscribe','rsquo',
                            'headline','lead story','body lead','view email','story body','copy paste',
                            'address window','web update','web page','receiving email','page go',
                            'go link','paste link','copy fact','receiving newsletter','anymore click',
                            'newsletter shown','newsletter clicking','interested anymore','emails view',
                            'html emails','software display','display html','please click','web updates',
                            'email','emails','interest','interested','newsletter'
                           ))
    #remove stop words
    new_str = ' '.join([word for word in str.split() if word not in cachedStopWords]) 
    return new_str

In [10]:
#Apply standardizing functions to body of email column
from nltk.corpus import stopwords

#0.) Convert everything to string
df['cleaned_body'] = df['body'].astype(str)
#1.) Lowercase
df.loc[:,'cleaned_body'] = df['cleaned_body'].str.lower()
#2.) Remove punctuation
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = punct)
#3.) Remove non alpha-numeric characters
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = remove_special_characters)
#4.) Remove digits
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = remove_digits)
#5.) Lemmatize words
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = lemmatize)
#6. Remove stop words
#df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
#7. Remove weird words that showed up in the analysis
df.loc[:,'cleaned_body'] = df['cleaned_body'].apply(func = remove_weird_words_that_showed_up_in_analysis_)

#Present results
df['cleaned_body'].head(5)


1    dear friend we have all seen the images and re...
2    team access to education is fundamental to a c...
3    beto for america john the trump administration...
4    john tomorrow is the fourth of july a day for ...
5    tomorrow is the fourth of july and donald trum...
Name: cleaned_body, dtype: object

In [11]:
df_dom = df[df['country']=="United States"]

DEM_df = df_dom[df_dom['party']=="Democratic"].reset_index(drop=True)
GOP_df = df_dom[df_dom['party']=="Republican"].reset_index(drop=True)

print(DEM_df.shape)
print(GOP_df.shape)

(27737, 19)
(6538, 19)


In [14]:
GOP_df1 = GOP_df[GOP_df['comp_score']==1]
GOP_df0 = GOP_df[GOP_df['comp_score']==0]

print('Positive Sentiment (GOP):', GOP_df1.shape)
print('Negative Sentiment (GOP):',GOP_df0.shape)

print('\n')
DEM_df1 = DEM_df[DEM_df['comp_score']==1]
DEM_df0 = DEM_df[DEM_df['comp_score']==0]

print('Positive Sentiment (DEM):',DEM_df1.shape)
print('Negative Sentiment (DEM):',DEM_df0.shape)

Positive Sentiment (GOP): (5638, 19)
Negative Sentiment (GOP): (900, 19)


Positive Sentiment (DEM): (24722, 19)
Negative Sentiment (DEM): (3015, 19)


In [15]:
#Full datasets
DEM = DEM_df.cleaned_body
GOP = GOP_df.cleaned_body

#GOP datasets broken out by sentiment
GOP1 = GOP_df1.cleaned_body
GOP0 = GOP_df0.cleaned_body

#DEM datasets broken out by sentiment
DEM1 = DEM_df1.cleaned_body
DEM0 = DEM_df0.cleaned_body

In [16]:
def clean_data(doc):
    doc = "".join(doc)
    doc = re.sub(r'[^\w\s]', '', doc)
    tokens = doc.split()
    #tokens = [word.lower() for word in tokens]
    tokens = [word for word in tokens if word.isalpha() ]
    #tokens = [word for word in tokens if not word in  stop_words]
    #text = " ".join(tokens)
    return tokens#, text

In [17]:
def return_lines(tokens,length):
    lines = []
    for i in range(length,len(tokens)):
        seq = tokens[i-length:i]
        line  = " ".join(seq)
        lines.append(line)
        
    return lines

In [18]:
#Full datasets
dem_tokens = clean_data(DEM)
gop_tokens = clean_data(GOP)

dem_lines = return_lines(dem_tokens,10)
gop_lines = return_lines(gop_tokens,10)

#DEM datasets
dem1_tokens = clean_data(DEM1)
dem0_tokens = clean_data(DEM0)

dem1_lines = return_lines(dem1_tokens,10)
dem0_lines = return_lines(dem0_tokens,10)

#GOP datasets
gop1_tokens = clean_data(GOP1)
gop0_tokens = clean_data(GOP0)

gop1_lines = return_lines(gop1_tokens,10)
gop0_lines = return_lines(gop0_tokens,10)

In [19]:
def prepare_data(data):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)
    sequences = np.array(sequences)
    X, y = sequences[:,:-1], sequences[:,-1]
    vocab_size = len(tokenizer.word_index) + 1
    return X,y,vocab_size,tokenizer

In [20]:
#Full datasets
dem_X,dem_y,dem_vocab_size,dem_tokenizer = prepare_data(dem_lines)
seq_dem = dem_X.shape[1] 

gop_X,gop_y,gop_vocab_size,gop_tokenizer = prepare_data(gop_lines)
seq_gop = gop_X.shape[1] 

#DEM datasets
dem1_X,dem1_y,dem1_vocab_size,dem1_tokenizer = prepare_data(dem1_lines)
seq_dem1 = dem1_X.shape[1] 

dem0_X,dem0_y,dem0_vocab_size,dem0_tokenizer = prepare_data(dem0_lines)
seq_dem0 = dem0_X.shape[1] 

#GOP datasets
gop1_X,gop1_y,gop1_vocab_size,gop1_tokenizer = prepare_data(gop1_lines)
seq_gop1 = gop1_X.shape[1] 

gop0_X,gop0_y,gop0_vocab_size,gop0_tokenizer = prepare_data(gop0_lines)
seq_gop0 = gop0_X.shape[1] 

In [21]:
def model_rnn(X,y,epochs,vocab_size,seq):
    model = keras.Sequential([keras.layers.Embedding(input_dim = vocab_size,output_dim=20,input_length=seq),
                          keras.layers.Bidirectional(keras.layers.LSTM(100)),
                          keras.layers.Dense(vocab_size,activation="softmax")])
    
    
    model.compile(loss="sparse_categorical_crossentropy",metrics="accuracy",optimizer="adam")
    early_stop = keras.callbacks.EarlyStopping(patience=5)
    
    model.fit(X,y,epochs=epochs,callbacks=early_stop)
    
    return model

In [36]:
def new_email(seed_text,num_word,model,seq):
    for _ in range(num_word):
        tokens = gop_tokenizer.texts_to_sequences([seed_text])[0]
        tokens = pad_sequences([tokens],maxlen=seq,padding="post")
    
        predicted = np.argmax(model.predict(tokens))
        outputword = ""
    
        for word,index in gop_tokenizer.word_index.items():
            if predicted == index:
                outputword = word
                break
            
        seed_text += " " + outputword
    return seed_text

### Generate Positive DEM Email

In [34]:
model = model_rnn(dem1_X,dem1_y,5,dem1_vocab_size,seq_dem1)

In [ ]:
seed_text = "republicans"
num_word = 50
new_email(seed_text,num_word,model,seq_dem1)

### Generate Negative DEM Email

In [33]:
model = model_rnn(dem0_X,dem0_y,15,dem0_vocab_size,seq_dem0)

Epoch 1/15
19502/19502 [==============================] - 137s 7ms/step - loss: 5.9217 - accuracy: 0.1281
Epoch 2/15
19502/19502 [==============================] - 129s 7ms/step - loss: 5.0845 - accuracy: 0.2024
Epoch 3/15
19502/19502 [==============================] - 126s 6ms/step - loss: 4.7201 - accuracy: 0.2349
Epoch 4/15
19502/19502 [==============================] - 126s 6ms/step - loss: 4.4810 - accuracy: 0.2557
Epoch 5/15
19502/19502 [==============================] - 128s 7ms/step - loss: 4.3042 - accuracy: 0.2707
Epoch 6/15
19502/19502 [==============================] - 125s 6ms/step - loss: 4.1649 - accuracy: 0.2823
Epoch 7/15
19502/19502 [==============================] - 129s 7ms/step - loss: 4.0557 - accuracy: 0.2910
Epoch 8/15
19502/19502 [==============================] - 125s 6ms/step - loss: 3.9670 - accuracy: 0.2982
Epoch 9/15
19502/19502 [==============================] - 129s 7ms/step - loss: 3.8894 - accuracy: 0.3054
Epoch 10/15
19502/19502 [=====================

In [35]:
seed_text = "republicans"
num_word = 50
new_email(seed_text,num_word,model,seq_dem0)

1/1 [==============================] - 0s 26ms/step


'republicans texasgopvote and and and and and and the did and i just county s is for so and here by lincoln single in the my s the agenda improve and a now all goal to america to the re of the did and the did it thepresident has to over'

### Generate Positive GOP Email

In [22]:
model = model_rnn(gop1_X,gop1_y,15,gop1_vocab_size,seq_gop1)

Epoch 1/15
40374/40374 [==============================] - 396s 10ms/step - loss: 5.7801 - accuracy: 0.1618
Epoch 2/15
40374/40374 [==============================] - 381s 9ms/step - loss: 4.8537 - accuracy: 0.2554
Epoch 3/15
40374/40374 [==============================] - 381s 9ms/step - loss: 4.5418 - accuracy: 0.2838
Epoch 4/15
40374/40374 [==============================] - 385s 10ms/step - loss: 4.3561 - accuracy: 0.2989
Epoch 5/15
40374/40374 [==============================] - 385s 10ms/step - loss: 4.2320 - accuracy: 0.3088
Epoch 6/15
40374/40374 [==============================] - 387s 10ms/step - loss: 4.1449 - accuracy: 0.3162
Epoch 7/15
40374/40374 [==============================] - 383s 9ms/step - loss: 4.0667 - accuracy: 0.3223
Epoch 8/15
40374/40374 [==============================] - 385s 10ms/step - loss: 4.0023 - accuracy: 0.3275
Epoch 9/15
40374/40374 [==============================] - 384s 10ms/step - loss: 3.9452 - accuracy: 0.3321
Epoch 10/15
40374/40374 [===============

In [25]:
seed_text = "democrats"
num_word = 50
new_email(seed_text,num_word,model,seq_gop1)

1/1 [==============================] - 0s 23ms/step


'democrats opioid opioid opioid opioid opioid opioid regional refusing beating in the re of the nancy united and the f s officials same of side and the side of the f s side and dependent i s right that what plan for your from to computer our keep and leading a'

### Generate Negative GOP Email

In [28]:
model = model_rnn(gop0_X,gop0_y,25,gop0_vocab_size,seq_gop0)

Epoch 1/25
6118/6118 [==============================] - 46s 7ms/step - loss: 6.5506 - accuracy: 0.0826
Epoch 2/25
6118/6118 [==============================] - 37s 6ms/step - loss: 5.5262 - accuracy: 0.1651
Epoch 3/25
6118/6118 [==============================] - 37s 6ms/step - loss: 4.9583 - accuracy: 0.2116
Epoch 4/25
6118/6118 [==============================] - 36s 6ms/step - loss: 4.5284 - accuracy: 0.2457
Epoch 5/25
6118/6118 [==============================] - 36s 6ms/step - loss: 4.1672 - accuracy: 0.2746
Epoch 6/25
6118/6118 [==============================] - 37s 6ms/step - loss: 3.8544 - accuracy: 0.3039
Epoch 7/25
6118/6118 [==============================] - 36s 6ms/step - loss: 3.5822 - accuracy: 0.3321
Epoch 8/25
6118/6118 [==============================] - 37s 6ms/step - loss: 3.3461 - accuracy: 0.3609
Epoch 9/25
6118/6118 [==============================] - 37s 6ms/step - loss: 3.1389 - accuracy: 0.3884
Epoch 10/25
6118/6118 [==============================] - 37s 6ms/step - l

In [29]:
seed_text = "democrats"
num_word = 50
new_email(seed_text,num_word,model,seq_gop0)

1/1 [==============================] - 0s 24ms/step


'democrats by by joined allowing in voted usmca management the take america the topic tricks did out great now within america father to the not and new learn grateful help ask today from advisor borders to the not of the so not m addiction illegal ll group in the please border'